## Importing Libraries & Datasets

In [21]:

#For Implementing Gramformer Solution
from gramformer import Gramformer

#for implementing Bert Solution
from happytransformer import HappyTextToText, TTSettings

#For Implementing LanguageTool Solution
import language_tool_python

#For GingerIt Solution
from gingerit.gingerit import GingerIt

#For Symspellpy
import pkg_resources
from symspellpy import SymSpell

#For TextBlob
from textblob import TextBlob

#For Web Application Development
import gradio as gr

import pandas as pd
import sys 


In [22]:
df = pd.read_csv('./grammatical error detection/NLP Assignment/test_data.csv')
df.head(20)
test_df = df.head(40)
pd.set_option('display.max_colwidth', None)

## Gramformer Model 

In [23]:
gf = Gramformer(models=1, use_gpu=False) #1=corrector, 2=detector

[Gramformer] Grammar error correct/highlight model loaded..


In [24]:
gf.correct('hello my dear childs')

{'hello my dear child.'}

    Function to build web application using gradio

In [25]:
def correct(sentence):
    res = gf.correct(sentence) 
    return res
# app_inputs = gr.inputs.Textbox(lines=3, placeholder="Enter a grammatically incorrect sentence here...")

# interface = gr.Interface(fn=correct, 
#                         inputs=app_inputs,
#                          outputs='text', 
#                         title='Hi there, I\'m Gramformer')

#interface.launch()

    Gramformer highlighter

In [26]:
# gh = Gramformer(models=3, use_gpu=False) #1=corrector, 2=detector

In [27]:
# gh.highlight(orig='to tha store',cor='to the store')

### Trying Gramformer on Datasets

In [28]:
def gramformer_corrector(text):
    res = gf.correct(text) 
    return res

    Calling Gramformer model for correction

In [29]:
# test_df['corrected_sentence'] = test_df['input'].apply(lambda text: gramformer_corrector(text))
# test_df.head(40)

## Bert + Huggingface Model 

In [30]:
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
args = TTSettings(num_beams=5, min_length=1)
result = happy_tt.generate_text("grammar: This sentences has has bads grammar.", args=args)
print(result.text)

This sentence has bad grammar.


In [31]:
def huggingface_corrector(text):
    result = happy_tt.generate_text('grammar: ' + text, args=TTSettings(num_beams=1, min_length=1, max_length=100))
    return result.text
# app_hugginface_inputs = gr.inputs.Textbox(lines=3, placeholder="Enter a grammatically incorrect sentence here...")

# interface2 = gr.Interface(fn=huggingface_corrector, 
#                         inputs=app_hugginface_inputs,
#                         outputs='text', 
#                         title='Hi there, I\'m Huggingface')

#interface2.launch()

### Trying Bert + Hugging Face on Datasets

In [32]:
def huggingface_corrector(text):
    result = happy_tt.generate_text('grammar: ' + text, args=TTSettings(num_beams=1, min_length=1, max_length=100))
    return result.text

    Calling Bert model for correction

In [33]:
# test_df['corrected_sentence'] = test_df['input'].apply(lambda text: huggingface_corrector(text))
# test_df.head(40)

## Language Tool Model

In [34]:
my_tool = language_tool_python.LanguageTool('en-US')  
my_text = """LanguageTool provides utility to check grammar and spelling errors. We just have to paste the text here and click the 'Check Text' button. Click the colored phrases for for information on potential errors. or we can use this text too see an some of the issues that LanguageTool can dedect. Whot do someone thinks of grammar checkers? Please not that they are not perfect. Style problems get a blue marker: It is 7 P.M. in the evening. The weather was nice on Monday, 22 November 2021"""   

def english_text_corrector(tool, text):
    
    matches = tool.check(text)

    #empty lists
    Mistakes = [] 
    Corrections = []  
    StartPositions = []  
    EndPositions = []  

    for rules in matches:
        if len(rules.replacements) > 0:  
            StartPositions.append(rules.offset)  
            EndPositions.append(rules.errorLength + rules.offset)  
            Mistakes.append(my_text[rules.offset : rules.errorLength + rules.offset])  
            Corrections.append(rules.replacements[0]) 

    print("Mistakes made")
    print (Mistakes)
    print ("\nRecommended Corrections")
    print(Corrections)
    print ("\nMistake Starting character number")
    print(StartPositions)
    print ("\nMistake EndPoint character number")
    print(EndPositions)

    mistakes_number = len (Mistakes)

    print( "\nNumber of mistakes made " + str(mistakes_number))
    #return mistakes_number

#english_text_corrector(my_tool, my_text)

### Language Tool Model on Datasets

In [35]:
tool = language_tool_python.LanguageTool('en-US')
def language_tool_corrector(text:str):
    correction = tool.correct(text)
    return correction

    Calling language tool model for correction

In [36]:
# test_df['corrected_sentence'] = test_df['input'].apply(lambda text: language_tool_corrector(text))
# test_df.head(40)

## GingerIt Model

In [37]:
text = 'The smelt of fliwers bring back memories.'

parser = GingerIt()
parser.parse(text)['result']

'The smell of flowers brings back memories.'

### Trying GingerIt Model on Datasets

In [38]:
def ginger_corrector(text: str):
    
    parser = GingerIt()
    correction = parser.parse(text)['result']
    return correction

        Calling GingerIt model for correction

In [39]:
# test_df['corrected_sentence'] = test_df['input'].apply(lambda text: ginger_corrector(text))
# test_df.head(40)

## Symspellpy Model

In [40]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt"
)
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)
input_term = (
    "whereis th elove hehad dated forImuch of thepast who "
    "couqdn'tread in sixtgrade and ins pired him"
)
# max edit distance per lookup (per single word, not per whole input string)
suggestions = sym_spell.lookup_compound(input_term, max_edit_distance=2)

for suggestion in suggestions:
    print(suggestion)

where is the love he had dated for much of the past who couldn't read in six grade and inspired him, 9, 0


In [41]:
def symspelly_corrector(text):
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt"
    )
    bigram_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_bigramdictionary_en_243_342.txt"
    )

    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)
    suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)

    for suggestion in suggestions:
        return suggestion

In [42]:
# test_df['corrected_sentence'] = test_df['input'].apply(lambda text: symspelly_corrector(text))
# test_df.head(40)

## Textblob

In [43]:
def textblob_corrector(text):
    correction = TextBlob(text)
    return correction.correct()

In [44]:
# test_df['ginger_corrected_sentence'] = test_df['input'].apply(lambda text: textblob_corrector(text))
# test_df.head()

## Combining GingerIt with Bert

In [45]:
# test_df['ginger_corrected_sentence'] = test_df['input'].apply(lambda text: ginger_corrector(text))
# test_df['combined_with_bert_corrected_sentence'] = test_df['ginger_corrected_sentence'].apply(lambda text: huggingface_corrector(text))
# test_df.head(40)

## Combining Gingerit with Gramformer

In [46]:
# test_df['ginger_corrected_sentence'] = test_df['input'].apply(lambda text: ginger_corrector(text))
# test_df['combined_with_gramformer'] = test_df['ginger_corrected_sentence'].apply(lambda text: gramformer_corrector(text))
# test_df.head(40)

# Testing

In [47]:
grammar_df = pd.read_csv("./grammar_error_check.csv")

In [48]:
grammar_df.head()

,error,real_correct,predicted_correct,Unnamed: 3,diff_r,diff_p
0,D 'Souza is amaced at the recognitaon eancing now has in India -- and the money involked .,D 'Souza is amazed at the recognition dancing now has in India -- and the money involved .,"D'Souza is amazed at the recognition, dancing now has in India -- and the money invoked.",NaN,"{'recognition', 'amazed', 'involved', 'dancing'}","{'recognition,', 'amazed', 'invoked.', 'dancing', ""D'Souza""}"
1,"Similarly , the United States gav Ge0rgia military aid in 2002 and 2003 to gelp mprove caunterterror capabilities .","Similarly , the United States gave Georgia military aid in 2002 and 2003 to help improve counterterror capabilities .","Similarly, the United States gave Georgia military aid in 2002 and 2003 to help improve its combat capabilities.",NaN,"{'help', 'counterterror', 'improve', 'Georgia', 'gave'}","{'help', 'improve', 'combat', 'capabilities.', 'Georgia', 'its', 'Similarly,', 'gave'}"
2,Others fear the neighbourhood 's character will be irseparabey changed because of luxury housing .,Others fear the neighbourhood 's character will be irreparably changed because of luxury housing .,Others fear the neighborhood's character will be irreparabey changed because of luxury housing.,NaN,{'irreparably'},"{""neighborhood's"", 'irreparabey', 'housing.'}"
3,Goldman reported from New York .,Goldman reported from New York .,Goldman reported from New York.,NaN,set(),{'York.'}
4,"But there 's no oubt that Dudley , im he runs , would bring name familiarity to the rac given that he had two stints playing for the Trail Blazers , from 1993-97 and from 2001-03 .","But there 's no doubt that Dudley , if he runs , would bring name familiarity to the race given that he had two stints playing for the Trail Blazers , from 1993-97 and from 2001-03 .","But there's no doubt that Dudley, I'm he runs, would bring name familiarity to the race given that he had two stints playing for the Trail Blazers, from 1993-97 and from 2001-03.",NaN,"{'if', 'doubt', 'race'}","{""there's"", 'Dudley,', 'runs,', 'doubt', ""I'm"", '2001-03.', 'Blazers,', 'race'}"


In [49]:
grammar_df.drop(columns=['Unnamed: 3', 'diff_r', 'diff_p'],inplace=True)

In [50]:
grammar_df['corrected_sentence'] = grammar_df['error'].apply(lambda text: huggingface_corrector(text))

In [51]:
grammar_df.head(3)

,error,real_correct,predicted_correct,corrected_sentence
0,D 'Souza is amaced at the recognitaon eancing now has in India -- and the money involked .,D 'Souza is amazed at the recognition dancing now has in India -- and the money involved .,"D'Souza is amazed at the recognition, dancing now has in India -- and the money invoked.",D'Souza is happy at the recognition he now has in India -- and the money involved.
1,"Similarly , the United States gav Ge0rgia military aid in 2002 and 2003 to gelp mprove caunterterror capabilities .","Similarly , the United States gave Georgia military aid in 2002 and 2003 to help improve counterterror capabilities .","Similarly, the United States gave Georgia military aid in 2002 and 2003 to help improve its combat capabilities.","Similarly, the United States gave the United States military aid in 2002 and 2003 to improve its defense capabilities."
2,Others fear the neighbourhood 's character will be irseparabey changed because of luxury housing .,Others fear the neighbourhood 's character will be irreparably changed because of luxury housing .,Others fear the neighborhood's character will be irreparabey changed because of luxury housing.,Others fear the neighbourhood's character will be irreplaceable because of luxury housing.


In [72]:
data = pd.DataFrame(grammar_df)
data.to_csv('error_analysis.csv', index=False, mode='w')


'error,real_correct,predicted_correct,corrected_sentence\r\nD \'Souza is amaced at the recognitaon eancing now has in India -- and the money involked .,D \'Souza is amazed at the recognition dancing now has in India -- and the money involved .,"D\'Souza is amazed at the recognition, dancing now has in India -- and the money invoked.",D\'Souza is happy at the recognition he now has in India -- and the money involved.\r\n"Similarly , the United States gav Ge0rgia military aid in 2002 and 2003 to gelp mprove caunterterror capabilities .","Similarly , the United States gave Georgia military aid in 2002 and 2003 to help improve counterterror capabilities .","Similarly, the United States gave Georgia military aid in 2002 and 2003 to help improve its combat capabilities.","Similarly, the United States gave the United States military aid in 2002 and 2003 to improve its defense capabilities."\r\nOthers fear the neighbourhood \'s character will be irseparabey changed because of luxury housing .,

In [61]:
ls

 Volume in drive B is Personal
 Volume Serial Number is 86B5-0838

 Directory of b:\Bernard\Programming\english_correction_models

10/25/2022  02:28 PM    <DIR>          .
10/25/2022  11:44 AM    <DIR>          ..
10/25/2022  11:45 AM                71 .gitignore
10/25/2022  01:50 PM            15,663 english_corrector_models.ipynb
10/25/2022  02:28 PM             6,193 error analysis.xlsx
10/25/2022  01:48 PM            62,465 grammar_error_check.csv
10/25/2022  11:45 AM    <DIR>          grammatical error detection
10/25/2022  11:45 AM                27 README.md
10/25/2022  11:45 AM    <DIR>          spell error detection
               5 File(s)         84,419 bytes
               4 Dir(s)  86,259,576,832 bytes free
